In [25]:
!pip install rpy2 

In [18]:
import os
import pandas as pd
import time
from nltk.util import skipgrams
from nltk.lm import NgramCounter

In [19]:
import rpy2.robjects as robjects
from rpy2.robjects.packages import importr
import rpy2.robjects.packages as rpackages
from rpy2.robjects import StrVector, IntVector
utils = rpackages.importr('utils')
utils.chooseCRANmirror(ind=1) # select the first mirror in the list
r_libs = '/home/krzys/R/x86_64-pc-linux-gnu-library/4.1'# R libs
base = importr('base')
print(base._libPaths())
utils.install_packages("wordspace")
wordspace = importr("wordspace", lib_loc=r_libs)

R[write to console]: Instalowanie pakietu w ‘/home/krzys/R/x86_64-pc-linux-gnu-library/4.1’
(ponieważ ‘lib’ nie jest określony)



[1] "/home/krzys/R/x86_64-pc-linux-gnu-library/4.1"
[2] "/usr/local/lib/R/site-library"                
[3] "/usr/lib/R/site-library"                      
[4] "/usr/lib/R/library"                           



R[write to console]: próbowanie adresu URL 'https://cloud.r-project.org/src/contrib/wordspace_0.2-7.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 1840058 bytes (1.8 MB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R

g++ -std=gnu++14 -I"/usr/share/R/include" -DNDEBUG  -I'/home/krzys/R/x86_64-pc-linux-gnu-library/4.1/Rcpp/include'   -fopenmp -fpic  -g -O2 -ffile-prefix-map=/build/r-base-4A2Reg/r-base-4.1.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c RcppExports.cpp -o RcppExports.o
g++ -std=gnu++14 -I"/usr/share/R/include" -DNDEBUG  -I'/home/krzys/R/x86_64-pc-linux-gnu-library/4.1/Rcpp/include'   -fopenmp -fpic  -g -O2 -ffile-prefix-map=/build/r-base-4A2Reg/r-base-4.1.2=. -fstack-protector-strong -Wformat -Werror=format-security -Wdate-time -D_FORTIFY_SOURCE=2 -g  -c wordspace.cpp -o wordspace.o
g++ -std=gnu++14 -shared -L/usr/lib/R/lib -Wl,-Bsymbolic-functions -flto=auto -ffat-lto-objects -flto=auto -Wl,-z,relro -o wordspace.so RcppExports.o wordspace.o -fopenmp -L/usr/lib/R/lib -lR


installing to /home/krzys/R/x86_64-pc-linux-gnu-library/4.1/00LOCK-wordspace/00new/wordspace/libs
** R
** data
*** moving datasets to lazyload DB
** inst
** byte-compile and prepare package for lazy loading
** help
*** installing help indices
** building package indices
** installing vignettes
** testing if installed package can be loaded from temporary location
** checking absolute paths in shared objects and dynamic libraries
** testing if installed package can be loaded from final location
** testing if installed package keeps a record of temporary installation path
* DONE (wordspace)
R[write to console]: 

R[write to console]: 
R[write to console]: The downloaded source packages are in
	‘/tmp/RtmpYf2CJN/downloaded_packages’
R[write to console]: 
R[write to console]: 



In [20]:
# define terms we're interested in
socio_political_terms = ["civitas", "consilium", "consul", "dux", "gens", "hostis", "imperator", "jus", "labor", "natio", "nobilitas", "pontifex", "pontificium", "populus", "potestas", "regnum", "senatus", "sodes", "urbs"]

In [21]:
# prepare metadata
dir_in = os.path.join("/home/krzys/Kod/streamlit/voces/data/corpora/latinise_IT_lemmas/")
dir_in
files = os.listdir(os.path.join(dir_in))
files = [f for f in files[:] if "IT" in f]
metadata_df = pd.read_csv(os.path.join(dir_in, 'latinise_metadata.csv'), sep = ",")
metadata_df = metadata_df[metadata_df['id'].str.startswith("IT")]
metadata_df.head()

# split the corpus into subcorpora
first_date = min(metadata_df.date)
last_date = max(metadata_df.date)
print(first_date)
print(last_date)
size_interval = 500
n_intervals = round((last_date-first_date)/size_interval)
n_intervals
intervals = [None]*(n_intervals+1)
for t in range(n_intervals+1):
    #print(t)
    if t == 0:
        intervals[t] = first_date
    else:
        intervals[t] = intervals[t-1]+size_interval
    #print(intervals[t])
    
print(intervals)

metadata_df['time_interval'] = ""
for t in range(len(intervals)-1):
    print(t)
    print(range(intervals[t],intervals[t+1]))
    metadata_df_t = metadata_df.loc[metadata_df['date'].isin(range(intervals[t],intervals[t+1]))]
    print(metadata_df_t.date)
    metadata_df.loc[metadata_df['date'].isin(range(intervals[t],intervals[t+1])),'time_interval'] = intervals[t]
metadata_df

def convert_dates(sign, date0):

    if sign == "0":
        if date0 == 0:
            final_date = "+0000"
        elif date0 < 100:
            final_date = "+" + "00" + str(date0)
            #print("1-final_date", final_date)
        elif date0 < 1000:
            final_date = "+" + "0" + str(date0)
            #print("2-final_date", final_date)
        else:
            final_date = "+" + str(date0)
            #print("3-final_date", final_date)
    else:
        if date0 == 0:
            final_date = "+0000"
        elif date0 < 100:
            final_date = str(sign) + "00" + str(date0)
            #print("1-final_date", final_date)
        elif date0 < 1000:
            final_date = str(sign) + "0" + str(date0)
            #print("2-final_date", final_date)
        else:
            final_date = str(sign) + str(date0)
            #print("3-final_date", final_date)

    if final_date.startswith("+"):
        final_date = final_date.replace("+", "")
    return final_date

-450
2005
[-450, 50, 550, 1050, 1550, 2050]
0
range(-450, 50)
19      -9
34     -49
38      49
39     -45
42     -49
      ... 
635   -149
638   -107
642    -37
643    -37
649   -229
Name: date, Length: 91, dtype: int64
1
range(50, 550)
18     382
20     524
23     399
24     391
37     158
      ... 
683    116
684    116
685    116
686    116
687    533
Name: date, Length: 236, dtype: int64
2
range(550, 1050)
102    800
104    800
105    800
106    800
107    800
      ... 
601    833
609    598
628    950
634    550
636    550
Name: date, Length: 66, dtype: int64
3
range(1050, 1550)
21     1254
22     1254
25     1224
26     1150
28     1517
       ... 
624    1067
625    1110
626    1110
644    1501
646    1483
Name: date, Length: 178, dtype: int64
4
range(1550, 2050)
27     2000
33     1998
35     2001
36     1992
220    1998
       ... 
641    1685
647    1550
650    1555
651    1684
652    1684
Name: date, Length: 99, dtype: int64


In [22]:
# prepare the corpus
punctuation = ['.', ',', '...', ';', ':', '?']

In [16]:
corpus = list()
corpus_filenames = list()
files_corpus = metadata_df
for index, df_line in files_corpus.iterrows():
    #print("line:",df_line['id'], df_line['time_interval'])
    sign = "+"
    #print(df_line['date'])
    if df_line['date'] < 0:
        sign = "-"
    #print("date:", convert_dates(sign, abs(df_line['date'])))
    file_name = 'lat_'+str(convert_dates(sign, abs(df_line['date'])))+"_"+str(df_line['id'])+'.txt'
    #KN: missing files
    if os.path.isfile(os.path.join(dir_in, file_name)):
        file = open(os.path.join(dir_in, file_name), 'r')
        sentences_this_file = list()
        while True:
            line = file.readline().strip()
            if line != "":
                corpus.append([token.lower() for token in line.split(" ") if token not in punctuation]) #KN: lemmas to lowcase
                corpus_filenames.append(file_name) #KN: track the file name for the sentence
            # if line is empty end of file is reached
            if not line:
                break
        file.close()
        corpus.append(sentences_this_file)
    else:
        print(os.path.join(dir_in, file_name), " doesn't exist")

NameError: name 'metadata_df' is not defined

In [31]:
#Let's compute cooccurrence counts in the corpus.
window=2
skip=0 # defines distance if skipgram
ngrams = [ skipgrams(sent, window, skip) for sent in corpus ]
ngram_counts = NgramCounter(ngrams)

In [32]:
# get triples (target, feature, freq) from the ngram_counts
coocs = []
for node, freqs in ngram_counts[window].items(): # dict_items([(('ars',), FreqDist({'et': 149, 'sum': 129, ...}}))])
    #print(word)
    for cooc, freq  in freqs.items():
        triple = (node[0], cooc, freq) # node term, cooc, freq
        coocs.append(triple)
print(coocs[0:10])

[('ars', 'amatorius', 2), ('ars', 'populus', 1), ('ars', 'cieo', 1), ('ars', 'levis', 1), ('ars', 'rego', 2), ('ars', 'ferus', 2), ('ars', 'nec', 9), ('ars', 'sum', 129), ('ars', 'careo', 4), ('ars', 'excutio', 1)]


In [33]:
# convert cooc counts to matrix
coocs_df = pd.DataFrame(coocs, columns=["target", "feature", "score"])
coocs_df.head()

,target,feature,score
0,ars,amatorius,2
1,ars,populus,1
2,ars,cieo,1
3,ars,levis,1
4,ars,rego,2


In [34]:
# create dsm matrix
VObj = wordspace.dsm(target=StrVector(coocs_df["target"]),
                     feature=StrVector(coocs_df["feature"]),
                     score=IntVector(coocs_df["score"]),
                     raw_freq=True) 
print(VObj)

Distributional Semantic Model with 192519 rows x 196956 columns
* raw co-occurrence matrix M available
  - sparse matrix with 2779.5k / 37.9G nonzero entries (fill rate = 0.01%)
  - in canonical format
  - known to be non-negative
  - sample size of underlying corpus: 7832.7k tokens



In [35]:
#default options
config = dict(score="simple-ll", transform="log", normalize=True, method="euclidean", reduce_method="svd", reduce_n=300)
print(config)
VObj_weighted = wordspace.dsm_score(VObj, score=config["score"], transform=config["transform"], normalize=config["normalize"], method=config["method"])
print(VObj_weighted)

{'score': 'simple-ll', 'transform': 'log', 'normalize': True, 'method': 'euclidean', 'reduce_method': 'svd', 'reduce_n': 300}
Distributional Semantic Model with 192519 rows x 196956 columns
* raw co-occurrence matrix M available
  - sparse matrix with 2779.5k / 37.9G nonzero entries (fill rate = 0.01%)
  - in canonical format
  - known to be non-negative
  - sample size of underlying corpus: 7832.7k tokens
* scored matrix S available
  - sparse matrix with 2500.3k / 37.9G nonzero entries (fill rate = 0.01%)
  - in canonical format
  - known to be non-negative



In [36]:
VObj_weighted_reduced = wordspace.dsm_projection(VObj_weighted, method=config["reduce_method"], n=config["reduce_n"])
print(VObj_weighted_reduced[0]) # takes around 5 mins

0.03863485012165868


In [37]:
# save matrices to disk
matrices = [VObj_weighted, VObj_weighted_reduced]
import pickle
f = open('dsm_matrices.model', 'wb')
pickle.dump(matrices,f)

In [ ]:
# get similar from unreduced space
[ print(term, "\n", wordspace.nearest_neighbours(VObj_weighted, term, n=10)) for term in socio_political_terms ]
# get similar from reduced space
[ print(term, "\n", wordspace.nearest_neighbours(VObj_weighted_reduced, term, n=10)) for term in socio_political_terms ]

In [23]:
# dictionary that maps a time interval with the list of sentences of texts in that time interval"
time2corpus = dict()

# I loop over all time intervals:
for t in range(n_intervals+1):
    files_corpus_t = metadata_df.loc[metadata_df['time_interval'] == intervals[t]]
    #print("1:",files_corpus_t, type(files_corpus_t))
    corpus_t = list()
    for index, df_line in files_corpus_t.iterrows():
        #print("line:",df_line['id'], df_line['time_interval'])
        sign = "+"
        #print(df_line['date'])
        if df_line['date'] < 0:
            sign = "-"
        #print("date:", convert_dates(sign, abs(df_line['date'])))
        file_name = 'lat_'+str(convert_dates(sign, abs(df_line['date'])))+"_"+str(df_line['id'])+'.txt'
        #print("3:",file_name)
        #KN: missing files
        if os.path.isfile(os.path.join(dir_in, file_name)):
            file = open(os.path.join(dir_in, file_name), 'r')
            sentences_this_file = list()
            while True:
                line = file.readline().strip()
                if line != "":
                    #sentences_this_file.append(line.split(" "))
                    #sentences_this_file.append([token for token in line.split(" ") if token not in punctuation])
                    corpus_t.append([token.lower() for token in line.split(" ") if token not in punctuation]) #KN: tolower
                # if line is empty end of file is reached
                if not line:
                    break
            file.close()
        #corpus_t.append(sentences_this_file)
    #corpus_t1
    #print(len(corpus_t1[0]))
    time2corpus[t] = corpus_t

In [24]:
time2corpus[4][0:2]

[['dominus',
  'iesus',
  'antequam',
  'in',
  'coelum',
  'ascendo',
  'suus',
  'discipulus',
  'mandatum',
  'confero',
  'nuntio',
  'evangelium',
  'cunctus',
  'homo',
  'omnesque',
  'populus',
  'baptizo',
  'eo',
  'in',
  'mundum',
  'universus',
  'praedico#1',
  'evangelium',
  'omnis',
  'creatura'],
 ['qui',
  'credo',
  'et',
  'baptizo',
  'sum',
  'salvus',
  'sum',
  'quis#2',
  'verus',
  'non',
  'credo',
  'condemno',
  '(',
  'mc',
  '-',
  ')',
  'data',
  'sum',
  'ego',
  'omnis',
  'potestas',
  'in',
  'caelum',
  'et',
  'in',
  'terra']]

In [41]:
window=2
skip=0 # defines distance if skipgram
# DSM options
config = dict(score="simple-ll", transform="log", normalize=True, method="euclidean", reduce_method="svd", reduce_n=300)
models = dict.fromkeys(range(len(time2corpus)))
models

{0: None, 1: None, 2: None, 3: None, 4: None, 5: None}

In [ ]:
start = time.time()
for t in range(len(time2corpus)):
    if len(time2corpus[t])>0:
        print("Building model for t = ", t, "\n")
        
        print("Retrieving coocurrence counts for t = ", t, "\n")
        #prepare coocs
        ngrams = [ skipgrams(sent, window, skip) for sent in time2corpus[t] ]
        ngram_counts = NgramCounter(ngrams)
        
        print("Retrieving (target, feature, freq) from the ngram_counts for t = ", t, "\n")
        # get triples (target, feature, freq) from the ngram_counts
        coocs = []
        for node, freqs in ngram_counts[window].items(): # dict_items([(('ars',), FreqDist({'et': 149, 'sum': 129, ...}}))])
            for cooc, freq  in freqs.items():
                triple = (node[0], cooc, freq) # node term, cooc, freq
                coocs.append(triple)
        
        coocs_df = pd.DataFrame(coocs, columns=["target", "feature", "score"])
        #coocs_df.head()
        
        print("Building DSM matrices for t = ", t, "\n")

        # create dms matrix
        VObj = wordspace.dsm(target=StrVector(coocs_df["target"]),
                             feature=StrVector(coocs_df["feature"]),
                             score=IntVector(coocs_df["score"]),
                             raw_freq=True) 
        VObj_weighted = wordspace.dsm_score(VObj, score=config["score"], transform=config["transform"], normalize=config["normalize"], method=config["method"])
        VObj_weighted_reduced = wordspace.dsm_projection(VObj_weighted, method=config["reduce_method"], n=config["reduce_n"])
        
        print("Saving models for t = ", t, "\n")
        models[t] = (VObj_weighted, VObj_weighted_reduced)
        
end = time.time()
print("It has taken", round(end - start), "seconds, or ", round((end - start)/60), "minutes")

In [43]:
# save diachronic models to disk
import pickle
f = open('dsm_matrices_time.model', 'wb')
pickle.dump(models,f)

In [44]:
# load models from disk
#import pickle
#f = open('dsm_matrices_time.model', 'rb')
#models = pickle.load(f)

In [ ]:
# get similarity counts
for t, model in models.items():
    if model is not None:
        print("Similarities for t =", t)
        print("Unreduced matrix for t =", t)    
        [ print(term, "\n", wordspace.nearest_neighbours(model[0], term, n=10) 
                if term in list(models[t][0].rx2('rows').rx2("term")) else None)
         for term in socio_political_terms ]
        print("Reduced matrix for t =", t)
        [ print(term, "\n", wordspace.nearest_neighbours(model[1], term, n=10)  
                if term in list(models[t][0].rx2('rows').rx2("term")) else None) 
         for term in socio_political_terms ]

# Experimenting with DSM options

In [25]:
from itertools import product
from nltk.util import everygrams, skipgrams
from nltk.util import ngrams
windows = range(2,5+1)
scores = [ "MI", "log-likelihood", "simple-ll", "t-score", "chi-squared", "z-score", "tf.idf" ]
transforms = [ "none", "log", "root", "sigmoid" ]
proj_methods = [ "svd", "rsvd" ]
threshold = [0, 3, 5, 10]
vectors = [50, 100, 300]
periods = [3] # test on 1 slice
normalizes = [ True, False ]
configs = []
[ configs.append({"window":x[0], "score":x[1], "transform":x[2], 
                  "min_f":x[3], "size":x[4], "period":x[5], "method":x[6], "normalize":x[7] }) 
 for x in product(windows, scores, transforms, threshold, vectors, periods, proj_methods, normalizes) ]
configs = []


In [26]:
# dict for ngram counts {period: {win1, win2...}}
ngram_counts_all = dict.fromkeys( k for k, v in time2corpus.items() )
ngram_counts_all

{0: None, 1: None, 2: None, 3: None, 4: None, 5: None}

In [27]:
for period in periods:
    print("Retrieving coocurrence counts for t = ", period, ", windows=", windows, "\n")
    #prepare coocs
    skip = max(windows) - 2
    ngrams = [ skipgrams(sent, 2, skip) for sent in time2corpus[period] ]
    len(ngrams)
    #print(ngrams)
    ngram_counts = NgramCounter(ngrams)
    len(ngram_counts)
    ngram_counts_all[period] = ngram_counts

Retrieving coocurrence counts for t =  3 , windows= range(2, 6) 



In [28]:
ngram_counts_all

{0: None,
 1: None,
 2: None,
 3: <nltk.lm.counter.NgramCounter at 0x7f5916be0550>,
 4: None,
 5: None}

In [ ]:
#[ a for a in ngram_counts_all[3] ]

In [30]:
models = list()

start = time.time()
for config in configs:
    print("model configuration: ", config)
    if len(time2corpus[config['period']]) > 0:        
        
        print("Building model for t = ", config['period'], "\n")
        
        print("Retrieving (target, feature, freq) from the ngram_counts for t = ", config['period'], "\n")
        # get triples (target, feature, freq) from the ngram_counts
        coocs = []
        ngram_counts = ngram_counts_all[config['period']]
        for node, freqs in ngram_counts[config['period']].items(): # dict_items([(('ars',), FreqDist({'et': 149, 'sum': 129, ...}}))])
            for cooc, freq  in freqs.items():
                triple = (node[0], cooc, freq) # node term, cooc, freq
                coocs.append(triple)
        print(coocs)
        coocs_df = pd.DataFrame(coocs, columns=["target", "feature", "score"])
        print(coocs_df.head())
        
        print("Building DSM matrices for t = ", config['period'], "\n")

        # create dms matrix
        VObj = wordspace.dsm(target=StrVector(coocs_df["target"]),
                             feature=StrVector(coocs_df["feature"]),
                             score=IntVector(coocs_df["score"]),
                             raw_freq=True) 
        
        VObj_weighted = wordspace.dsm_score(VObj, score=config["score"], transform=config["transform"],
                                            normalize=config["normalize"], method=config["method"])
        VObj_weighted_reduced = wordspace.dsm_projection(VObj_weighted, method=config["method"], n=config["min_f"])
        
        print("Saving models for t = ", config['period'], "\n")
        models.append((config, VObj_weighted, VObj_weighted_reduced))
        
end = time.time()
print("It has taken", round(end - start), "seconds, or ", round((end - start)/60), "minutes")
        

It has taken 0 seconds, or  0 minutes
